In [47]:
import feedparser
from lxml import html
import requests
import sqlite3
import time
import numpy as np
from functions import *
from bs4 import BeautifulSoup
import re, requests

In [5]:
url = "http://vancouver.craigslist.ca/search/apa?format=rss"
apts = feedparser.parse( url )
conn = sqlite3.connect('apartments.db')
c = conn.cursor()

In [186]:
def parse_int(string):
    return int(re.sub('[^0-9]', '', string))
               
def get_title(soup):
    title= soup.find_all(id='titletextonly')[0]
    return title.find(text=True, recursive=False).strip()

def get_bedrooms(soup):
    soup.select('.housing')[0]
    
def get_price(soup):
    item = soup.select('.price')[0]
    price_string = item.find(text=True, recursive=False).strip()
    return int(re.sub('[^0-9]', '', price_string))

def get_location(tree):
    longitude = float(tree.xpath('//*[@id="map"]//@data-longitude')[0])
    latitude = float(tree.xpath('//*[@id="map"]//@data-latitude')[0])
    return [latitude,longitude]

def get_address(soup):
    return soup.select('.mapaddress')[0].find(text=True, recursive=False).strip()

def get_location_soup(soup):
    # get the location using soup
    div = soup.find_all(id="map")[0]
    longitude = 0
    latitude = 0
    return [latitude,longitude]

def get_bedrooms(soup):
    attr = soup.select('.attrgroup')[0]
    bedrooms = None
    for s in attr.strings:
        if s.find("BR") != -1:
            bedrooms = parse_int(s)
    return bedrooms

def get_bathrooms(soup):
    attr = soup.select('.attrgroup')[0]
    bathrooms = None
    for s in attr.strings:
        if s.find("Ba") != -1:
            text = s.find("Ba")
            bathrooms = float(s[:text])
    return bathrooms

def get_area(soup):
    attr = soup.select('.attrgroup')[0]
    area = None
    for s in attr.strings:
        if s.find("ft") != -1:
            area = parse_int(s.find(text=True, recursive=False).strip())
    return area

def get_all_the_stuff(soup):
    attrs = soup.select('.attrgroup')
    stuff = []
    for attr in attrs:
        for s in attr.strings:
            if s.strip() !='':
                stuff.append(s.strip())
    return stuff

In [158]:
entry = apts.entries[2]
page = requests.get(entry.link)
tree = html.fromstring(page.content)
soup = BeautifulSoup(page.text, "html.parser")
print(entry.link)
get_location(tree)

http://vancouver.craigslist.ca/van/apa/6168162921.html


[49.271707, -123.167569]

In [159]:
get_title(soup)

'View of a Lifetime'

In [160]:
get_price(soup)

4800

In [162]:
get_address(soup)

'Point Grey Road'

In [163]:
get_location(tree)

[49.271707, -123.167569]

In [172]:
get_bedrooms(soup)

3

In [187]:
get_bathrooms(soup)

1.5

In [188]:
get_all_the_stuff(soup)

['3BR',
 '/',
 '1.5Ba',
 '1400',
 'ft',
 '2',
 'available aug 1',
 'house',
 'w/d in unit',
 'no smoking',
 'off-street parking']

'936 East 40th Ave'